<a href="https://colab.research.google.com/github/wassimchouchen/Automatic-Speech-Recognition-/blob/main/Transcription%26Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null
!pip install pydub
import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding( 
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib
import pydub

from sklearn.cluster import AgglomerativeClustering
import numpy as np


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
num_speakers = 2 
language = 'any' 
model_size = 'base'

In [3]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

Saving Conversation gynécologue patient 2.m4a to Conversation gynécologue patient 2 (2).m4a


In [4]:
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'
# Load the audio file
audio_file = pydub.AudioSegment.from_wav(path)
audio_file = audio_file.set_channels(1)
audio_file.export(path, format="wav")
with wave.open(path) as f:
  print(f.getnchannels())

1


In [5]:
model = whisper.load_model(model_size)

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 61.0MiB/s]


In [6]:
result = model.transcribe(path)
segments = result["segments"]

In [7]:
segments

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 4.0,
  'text': " Alors c'était quand même la petite !",
  'tokens': [50364, 9946, 269, 6, 9743, 6932, 5698, 635, 18319, 2298, 50564],
  'temperature': 0.0,
  'avg_logprob': -0.7748784069766366,
  'compression_ratio': 1.7267605633802816,
  'no_speech_prob': 0.5249548554420471},
 {'id': 1,
  'seek': 0,
  'start': 4.0,
  'end': 6.0,
  'text': ' Euh, bien non je crois.',
  'tokens': [50564, 47320, 11, 3610, 2107, 1506, 21724, 13, 50664],
  'temperature': 0.0,
  'avg_logprob': -0.7748784069766366,
  'compression_ratio': 1.7267605633802816,
  'no_speech_prob': 0.5249548554420471},
 {'id': 2,
  'seek': 0,
  'start': 6.0,
  'end': 9.0,
  'text': " Comment ça se croit qu'il te faudrait tirer ma grande a et alors c'était temps ?",
  'tokens': [50664,
   16328,
   2788,
   369,
   4848,
   270,
   421,
   6,
   388,
   535,
   38694,
   8645,
   13807,
   260,
   463,
   8883,
   257,
   1030,
   11246,
   269,
   6,
   9743,
   8827,
   2506,
   5

In [8]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [9]:
audio = Audio()
import pyannote.audio.utils

# Convert the audio recording to mono
def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [10]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [11]:
from sklearn.cluster import KMeans
clustering = KMeans(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [12]:
segments

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 4.0,
  'text': " Alors c'était quand même la petite !",
  'tokens': [50364, 9946, 269, 6, 9743, 6932, 5698, 635, 18319, 2298, 50564],
  'temperature': 0.0,
  'avg_logprob': -0.7748784069766366,
  'compression_ratio': 1.7267605633802816,
  'no_speech_prob': 0.5249548554420471,
  'speaker': 'SPEAKER 1'},
 {'id': 1,
  'seek': 0,
  'start': 4.0,
  'end': 6.0,
  'text': ' Euh, bien non je crois.',
  'tokens': [50564, 47320, 11, 3610, 2107, 1506, 21724, 13, 50664],
  'temperature': 0.0,
  'avg_logprob': -0.7748784069766366,
  'compression_ratio': 1.7267605633802816,
  'no_speech_prob': 0.5249548554420471,
  'speaker': 'SPEAKER 1'},
 {'id': 2,
  'seek': 0,
  'start': 6.0,
  'end': 9.0,
  'text': " Comment ça se croit qu'il te faudrait tirer ma grande a et alors c'était temps ?",
  'tokens': [50664,
   16328,
   2788,
   369,
   4848,
   270,
   421,
   6,
   388,
   535,
   38694,
   8645,
   13807,
   260,
   463,
   8883,
   257,
   1030,
   

In [13]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w",encoding="utf-8")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:])
f.close()

In [14]:
print(open("transcript.txt", "r", encoding="utf-8").read())


SPEAKER 1 0:00:00
Alors c'était quand même la petite !Euh, bien non je crois.
SPEAKER 2 0:00:06
Comment ça se croit qu'il te faudrait tirer ma grande a et alors c'était temps ?Oui, détendez vous !Oui oui, ben t'es détendré tout !
SPEAKER 1 0:00:11
Ouais, ouais, ouais, ouais.Allez j'y vais !
SPEAKER 2 0:00:13
Hop !J'ai tombé à Dieu mon ventre dehors hein !
SPEAKER 1 0:00:15
Ah oui ?
SPEAKER 2 0:00:16
Bah oui, ou alors qu'il y a des massaries de bien-licose ?Bon c'est pas grand chose hein, je vous donne de quoi la soigner ?On ne faut pas vous en faire, mais ça se fait déjà bien !C'est parce que vous êtes déjà introservés des bachakis comme !
SPEAKER 1 0:00:25
Oui.
SPEAKER 2 0:00:26
Oui bah oui, des tangas, des trinkis comme !Voilà, ça se sert à l'édorien hein !C'est déjà dit à quand on, quand on, quand on !Hein ? C'est comme une salle de bras après une douche hein !J'arrive au que ça respire hein !Des jus, pépis !On fait passer le vol à dessous quoi !
SPEAKER 1 0:00:39
Voilà, hop !
SPEA